# An example of parameters estimation using supernovas and Chaplygin gas


This is just a fancy way to visualize data 

In [ ]:
def PrettyPrint(data):
    from prettytable import PrettyTable
    x = PrettyTable(data.dtype.names)    
    for row in data:
        x.add_row(row)
    print(x)

Luminosity distance is defined in  terms of an integral that depends on the cosmological parameters that want to be estimated. If we define the function $$E(z) = \left[ (1-\Omega_{m0}) + \Omega_{m0}(1+z)^{3(1+\alpha)} \right]^{1/2(1+\alpha)},$$ the luminosity distance is given by: $$d_{L} = \frac{3\times 10^{3}}{h}(1+z)\int_{0}^{z}\frac{dz'}{E(z')}$$

In [ ]:
def Integrand(z, parameters):
    import numpy as np
    E =  np.power( (1 - parameters[1]) + parameters[1] * np.power(1 + z, 3*(1+parameters[2])), 0.5*(1+parameters[2]) ) 
    return 1. / E

In [ ]:
def dl(z, parameters):
    import numpy as np
    from scipy.integrate import quad
    I,e = quad(Integrand, 0, z,  args=(parameters))
    return 3000 * (1 + z) * I / parameters[0]

The module distance is the quantity that is related with the experimental data, because we do not measure distance directly. From the definitions of apparent and absolute magnitude  we have. $$\mu = m-M = 5\log{d_{L}}+25$$

In [ ]:
def mu(dl):
    import numpy as np
    return 5 * np.log10(dl) + 25

And we define our $\chi^{2}$ test respect to this quantity. So we want to find the parameters that minimizes 
$$\chi^{2}= \sum \left(\frac{\mu_{teo}-\mu_{exp}}{\sigma_{mu}}\right)^{2} $$ 

In [ ]:
def chi2(mut, muobs,  sigma):
    import numpy as np
    aux = 0
    for i in range(len(mut)):
        #aux += np.power( (mut[i] - muobs[i]) / sigma[i], 2)
        aux += np.power((mut[i] - muobs[i]), 2)/( np.power( sigma[i], 2) + 0.0169)
    return aux

This function performs a run with two free parameters: $h$ and $\Omega_{m0}$. It is not optimized, so it will run over a full set of values defined uniformly for each parameter. The first three args correspond to the number of divisions, the min value and the maximum value for the range of the $h$ parameter. In the same way are defined the following three arguments.

In [ ]:
def runOmAlpha(steps1, minval1,  maxval1, steps2, minval2, maxval2, data,pars):
    import numpy as np
    import itertools as it
    import pandas as pd
    muobs = data['m-M']
    sigma =  data['sigma']
    muteo = [None] * len(muobs)
   
    
    #Output data
    keys =  ["h", "Om", "alpha", "chi2"]
    outdata = { key : [None] for key in keys }  
    delta1 =  (maxval1 - minval1) / steps1
    delta2 =  (maxval2 - minval2) / steps2
    for i, j in it.product(range(steps1),  range(steps2)) :
        pars[1] = minval1 + delta1* i
        pars[2] = minval2 + delta2*j
        for k in range(len(data['z'])):
            muteo[k] =  mu( dl(data['z'][k] ,  pars) )
        chiaux = chi2(muteo, muobs, sigma)
        outdata["h"].append(pars[0])
        outdata["Om"].append(pars[1])
        outdata["Alpha"].append(pars[2])
        outdata["chi2"].append(chiaux)
        #print (Omegas[0] ," ",Omegas[1] ," ",h , " ", chiaux)

    df = pd.DataFrame(outdata)
    #print(df)
    boolean =  (df['chi2'] == np.min(df['chi2']))
    print ('h =', list(df.loc[boolean, 'h'])[0] ,  '\n' )
    print ('Omega_m =', list(df.loc[boolean, 'Om'])[0] ,  '\n' )
    print ('Alpha =', list(df.loc[boolean, 'alpha'])[0] ,  '\n' )
    print ('chi2 =', list(df.loc[boolean, 'chi2'])[0] ,  '\n' )
    print ('reduced_chi2 =', list(df.loc[boolean, 'chi2'])[0] / (len(data) - 2) ,  '\n' )
 

In [ ]:
import numpy as np
data = np.loadtxt("Legacy.dat",  dtype={'names': ('z', 'm-M' ,'sigma'), 'formats': ('f4', 'f4', 'f4')})
PrettyPrint(data)

In [ ]:
parameters=[0.7, None, None]
runOmAlpha(50, 0,  1, 100,  -1, 1, data,parameters)